# Retrieval Augmented Generation (RAG) with OpenAI and Qdrant

In the ever-evolving landscape of AI, the consistency and reliability of Large Language Models (LLMs) remain a challenge. While these models can understand statistical relationships between words, they often fail to provide accurate factual responses. Because their internal knowledge may not be accurate, outputs can range from spot-on to nonsensical. Retrieval Augmented Generation (RAG) is a framework designed to bolster the accuracy of LLMs by grounding them in external knowledge bases. In this example, we'll demonstrate a streamlined  implementation of the RAG pipeline using only Qdrant and OpenAI SDKs. By harnessing Flag embedding's power, we can bypass additional frameworks' overhead.
    
This example assumes you understand the architecture necessary to carry out RAG. If this is new to you, please look at some introductory readings:
* [Retrieval-Augmented Generation: To add knowledge](https://eugeneyan.com/writing/llm-patterns/#retrieval-augmented-generation-to-add-knowledge)

## Prerequisites

Let's start setting up all the pieces to implement the RAG pipeline. We will only use Qdrant and OpenAI SDKs, without any third-party libraries.

### Preparing the environment

We need just a few dependencies to implement the whole application, so let's start with installing the dependencies.

In [3]:
# To run in Google Colab, need to install these libraries, then restart session
!pip install cohere tiktoken protobuf==3.20.3 typing-extensions==4.5.0 qdrant-client fastembed openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 631.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of openai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 22.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.4 MB/

[Qdrant](https://qdrant.tech) will act as a knowledge base providing the context information for the prompts we'll be sending to the LLM. There are various ways of running Qdrant, but we'll simply use the Docker container.

In [3]:
# Docker
# !docker run -p "6333:6333" -p "6334:6334" --name "rag-openai-qdrant" --rm -d qdrant/qdrant:latest

### Creating the collection

Qdrant [collection](https://qdrant.tech/documentation/concepts/collections/) is the basic unit of organizing your data. Each collection is a named set of points (vectors with a payload) among which you can search. After connecting to our running Qdrant container, we can check whether we already have some collections.

In [4]:
import qdrant_client

# Docker
# client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
# client.get_collections()

# Memory
client = qdrant_client.QdrantClient(":memory:")
client.get_collections()

CollectionsResponse(collections=[])

### Building the knowledge base

Qdrant will use vector embeddings of our facts to enrich the original prompt with some context. Thus, we need to store the vector embeddings and the texts used to generate them. All our facts will have a JSON payload with a single attribute and look as follows:

```json
{
    "document": "Binary Quantization is a method of reducing the memory usage even up to 40 times!"
}
```

This structure is required by [FastEmbed](https://qdrant.github.io/fastembed/), a library that simplifies managing the vectors, as you don't have to calculate them on your own. It's also possible to use an existing collection, However, all the code snippets will assume this data structure. Adjust your examples to work with a different schema.

FastEmbed will automatically create the collection if it doesn't exist. Knowing that we are set to add our documents to a collection, which we'll call `knowledge-base`.

In [5]:
client.add(
    collection_name="knowledge-base",
    documents=[
        "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
        "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
        "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
        "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
        "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
        "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
        "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
        "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
    ]
)

['2c89bfaaefef45458c19e89ec5a754b7',
 'db756459c3104666959331df60b180e3',
 '0b2098bd6a7d45c1948e7d64e2426250',
 '76169f45f00a4cbeba4a4a855a37d98f',
 '81bfbe3426714333be14f909858d14c2',
 'c689ff057cef4c75ab9975f85a153195',
 '3861e3d0d98d43838a3df54e3cfd44f7',
 '5f47ddab6ac54abf82ccd019e813b330']

## Retrieval Augmented Generation

RAG changes the way we interact with Large Language Models. We're converting a knowledge-oriented task, in which the model may create a counterfactual answer, into a language-oriented task. The latter expects the model to extract meaningful information and generate an answer. LLMs, when implemented correctly, are supposed to be carrying out language-oriented tasks.

The task starts with the original prompt sent by the user. The same prompt is then vectorized and used as a search query for the most relevant facts. Those facts are combined with the original prompt to build a longer prompt containing more information.

But let's start simply by asking our question directly.

In [6]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

Using OpenAI API requires providing the API key. Our example demonstrates setting the `OPENAI_API_KEY` using an environmental variable.

In [7]:
import os
from google.colab import userdata

# Fill the environmental variable with your own Open Router API key
# See: https://openrouter.ai/keys
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Now we can finally call the completion service.

In [9]:
from openai import AsyncOpenAI

openai = AsyncOpenAI()
openai.base_url = "https://openrouter.ai/api/v1"

completion = await openai.chat.completions.create(
    model="mistralai/mistral-7b-instruct", # "gpt-3.5-turbo" # Cost too much
    messages=[
        {"role": "user", "content": prompt},
    ]
)

print(completion.choices)

[Choice(finish_reason=None, index=None, logprobs=None, message=ChatCompletionMessage(content='To build a web service using vector embeddings for search, you will need the following tools:\n\n1. A programming language: Python, Java, or any other language that supports building web services.\n2. A machine learning library: TensorFlow, PyTorch, or any other library that supports vector embeddings.\n3. A web framework: Flask, Django, or any other web framework that supports building web services.\n4. A database: MySQL, PostgreSQL, or any other database that supports indexing and querying.\n5. A text processing library: NLTK, spaCy, or any other library that supports text preprocessing and tokenization.\n6. A search engine: Elasticsearch, Solr, or any other search engine that supports vector embeddings.\n7. A cloud platform: AWS, Google Cloud, or any other cloud platform that supports deploying web services.', role='assistant', function_call=None, tool_calls=None))]


In [13]:
print(completion.choices[0].message.content)

To build a web service using vector embeddings for search, you will need the following tools:

1. A programming language: Python, Java, or any other language that supports building web services.
2. A machine learning library: TensorFlow, PyTorch, or any other library that supports vector embeddings.
3. A web framework: Flask, Django, or any other web framework that supports building web services.
4. A database: MySQL, PostgreSQL, or any other database that supports indexing and querying.
5. A text processing library: NLTK, spaCy, or any other library that supports text preprocessing and tokenization.
6. A search engine: Elasticsearch, Solr, or any other search engine that supports vector embeddings.
7. A cloud platform: AWS, Google Cloud, or any other cloud platform that supports deploying web services.


### Extending the prompt

Even though the original answer sounds credible, it didn't answer our question correctly. Instead, it gave us a generic description of an application stack. To improve the results, enriching the original prompt with the descriptions of the tools available seems like one of the possibilities. Let's use a semantic knowledge base to augment the prompt with the descriptions of different technologies!

In [14]:
results = client.query(
    collection_name="knowledge-base",
    query_text=prompt,
    limit=3,
)
results

[QueryResponse(id='2c89bfaaefef45458c19e89ec5a754b7', embedding=None, metadata={'document': 'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!'}, document='Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!', score=0.8717049365572785),
 QueryResponse(id='c689ff057cef4c75ab9975f85a153195', embedding=None, metadata={'document': 'FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.'}, document='FastAPI is a modern, fast

In [17]:
[result.metadata['document'] for result in results]

['Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!',
 'FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.',
 'PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.']

We used the original prompt to perform a semantic search over the set of tool descriptions. Now we can use these descriptions to augment the prompt and create more context.

In [19]:
context = "\n".join(r.document for r in results)
print(context)

Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.
PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.


Finally, let's build a metaprompt, the combination of the assumed role of the LLM, the original question, and the results from our semantic search that will force our LLM to use the provided context.

By doing this, we effectively convert the knowledge-oriented task into a language task and hopefully reduce the chances of hallucinations. It also should make the response sound more relevant.

In [20]:
metaprompt = f"""
You are a software architect.
Answer the following question using the provided context.
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: {prompt.strip()}

Context:
{context.strip()}

Answer:
"""

# Look at the full metaprompt
print(metaprompt)


You are a software architect. 
Answer the following question using the provided context. 
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: What tools should I need to use to build a web service using vector embeddings for search?

Context: 
Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.
PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.

Answer:



Our current prompt is much longer, and we also used a couple of strategies to make the responses even better:

1. The LLM has the role of software architect.
2. We provide more context to answer the question.
3. If the context contains no meaningful information, the model shouldn't make up an answer.

Let's find out if that works as expected.

In [21]:
completion = await openai.chat.completions.create(
    model="mistralai/mistral-7b-instruct",
    messages=[
        {"role": "user", "content": metaprompt},
    ],
    timeout=10.0,
)
print(completion.choices[0].message.content)

To build a web service using vector embeddings for search, you would need to use the following tools:

1. Qdrant: This is a vector database and search engine that allows you to store and search for high-dimensional vectors. It provides a REST API for interacting with the vector embeddings.
2. FastAPI: This is a modern and fast web framework for building APIs with Python. You can use FastAPI to create a web service that interacts with the Qdrant API and provides a user-friendly interface for searching and interacting with the vector embeddings.
3. PyTorch: This is a machine learning framework that can be used to train and deploy neural network encoders for generating vector embeddings. You can use PyTorch to train your own neural network encoders or to use pre-trained models for generating embeddings.

By using these tools together, you can build a powerful and scalable web service for search using vector embeddings.


### Testing out the RAG pipeline

By leveraging the semantic context we provided our model is doing a better job answering the question. Let's enclose the RAG as a function, so we can call it more easily for different prompts.

In [22]:
async def rag(question: str, n_points: int = 3) -> str:
    results = client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect.
    Answer the following question using the provided context.
    If you can't find the answer, do not pretend you know it, but answer "I don't know".

    Question: {question.strip()}

    Context:
    {context.strip()}

    Answer:
    """

    completion = await openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion.choices[0].message.content

Now it's easier to ask a broad range of questions.

In [23]:
await rag("What can the stack for a web api look like?")

'The stack for a web API using FastAPI, Docker, and NGINX could look like:\n\n1. NGINX: As a reverse proxy server, NGINX acts as the entry point for incoming requests to the API. It can handle load balancing, caching, SSL termination, and other features required for managing incoming traffic.\n\n2. FastAPI: FastAPI is used to build the actual API itself. It is a Python web framework that leverages type hints and asynchronous programming to provide high-performance and scalable APIs.\n\n3. Docker: Developers can use Docker to containerize their FastAPI application and dependencies. This ensures that the application runs consistently across different environments and can be easily deployed and scaled.\n\nSo, the stack for a web API can consist of NGINX as the reverse proxy, FastAPI as the web framework, and Docker for containerization.'

In [24]:
await rag("Where is the nearest grocery store?")

"I don't know."

Our model can now:

1. Take advantage of the knowledge in our vector datastore.
2. Answer, based on the provided context, that it can not provide an answer.

We have just shown a useful mechanism to mitigate the risks of hallucinations in Large Language Models.

### Cleaning up the environment

If you wish to continue playing with the RAG application we created, don't do the code below. However, it's always good to clean up the environment, so nothing is left dangling. We'll show you how to remove the Qdrant container.

In [ ]:
# !docker kill rag-openai-qdrant
# !docker rm rag-openai-qdrant

rag-openai-qdrant


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error response from daemon: No such container: rag-openai-qdrant


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
